In [1]:

from playwright.async_api import async_playwright
import asyncio
import nest_asyncio
from bs4 import BeautifulSoup
import psycopg2
from multiprocessing import Process, set_start_method
import multiprocessing
import time  # Import the time module
import gc
import nest_asyncio


async def get_detail(offset):
    async with async_playwright() as p:
      browser = await p.chromium.launch(headless=True)
      context = await browser.new_context()
      page = await context.new_page()
      url = "https://news.google.com/rss/articles/CBMieWh0dHBzOi8vd3d3Lndpb25ld3MuY29tL2luZGlhLW5ld3MvaW5kaWFuLWRlZmVuY2UtbWluaXN0ZXItaGludHMtY2hpbmEtbWlnaHQtd2VhcG9uaXNlLXdlYXRoZXItZm9yLXN0cmF0ZWdpYy1nYWlucy02ODE4NjbSAX1odHRwczovL3d3dy53aW9uZXdzLmNvbS9pbmRpYS1uZXdzL2luZGlhbi1kZWZlbmNlLW1pbmlzdGVyLWhpbnRzLWNoaW5hLW1pZ2h0LXdlYXBvbmlzZS13ZWF0aGVyLWZvci1zdHJhdGVnaWMtZ2FpbnMtNjgxODY2L2FtcA?oc=5"
      try:
          await page.goto(url, timeout=60000)
          await page.wait_for_load_state('load', timeout=60000)
          await asyncio.sleep(10)  # Add a delay here
      except:
          try:
              await page.wait_for_load_state('domcontentloaded', timeout=60000)
          except:
              print("Network error, retrying...")
      page_content = await page.content()
      soup = BeautifulSoup(page_content, 'html.parser')
      final_url = page.url
      title = ""
      detail = ""
      image_urls = []

      div = soup.find('div', attrs={'class': "article-heading"})
      if div:
          texts = [element.text for element in div.find_all(['h1', 'h2', 'h3'])]
          title = ' '.join(texts)
      main_div = soup.find('div', attrs={'class': "article-main-data"})
      if main_div:
          inner_divs = main_div.find_all('div', attrs={'class': None})
          for div in inner_divs:
              p_tags = div.find_all('p')
              for p in p_tags:
                  detail = detail + " " + p.text
      div = soup.find('div', attrs={'class': "article-main-img"})
      if div:
          source_media = div.find('img')
          if source_media:
              urls = source_media.get('srcset').split(',')
              last_url = urls[-1].split()[0]
              image_urls.append(last_url)

      print(final_url)
      print(title)
      print(detail)
      print(image_urls)
      await page.close()

def run_parallel(offset):
    asyncio.run(get_detail(offset))

if __name__ == "__main__":
    args = [0]
    processes = []
    for arg in args:
        process = multiprocessing.Process(target=run_parallel, args=(arg,))
        processes.append(process)
        process.start()

    for process in processes:
        process.join()